In [ ]:
# Loading in functions
import os
import csv
import torch
import pandas as pd
from PIL import Image
from os import listdir
from model import DogDetectorModel
from image_to_tensor import to_tensor
from non_max_surpression import NonMaxSurpression



def dog_extractor(image):

    # Initializing the images path
    img_path = "/Users/karthik/Desktop/open-images-v6/train/data/" + image

    # Initializing the model
    model = DogDetectorModel()

    # Initializing Non Max Surpression with default parameters
    nms = NonMaxSurpression()

    # Loading the checkpoint
    checkpoint = torch.load('model.pt', map_location=torch.device('cpu')) 

    # Loading the model weights
    model.load_state_dict(checkpoint['model_state_dict'])

    # Setting model to evaluation model
    model = model.eval()

    # Loading and transforming images
    images = to_tensor([img_path])

    # Running image through model
    outputs = model(images, '')

    # Executing non max surpression on the model output
    outputs = nms(outputs)

    return outputs

vals = []
folder_dir = "/Users/karthik/Desktop/open-images-v6/train/data/"
img_files = os.listdir(folder_dir)

for images in img_files:
    
    try:
        if (images.endswith(".png") or images.endswith(".jpg") or images.endswith(".jpeg")):

            # Get actual width and height
            img_path = "/Users/karthik/Desktop/open-images-v6/train/data/" + images
            im = Image.open(img_path)
            aw, ah = im.size

            # Output of the parameter
            output = dog_extractor(images)
            b_box = output[0]['boxes'][0]

            # Calculating features
            w = b_box[2] - b_box[0]
            h = b_box[3] - b_box[1]
            normed_w = w/aw
            normed_h = h/ah
            area = normed_w * normed_h
            
            features = {'image': images,
                        'a_width': aw,
                        'a_height': ah,
                        'width': w,
                        'height': h,
                        'normed_w': normed_w,
                        'normed_h': normed_h,
                        'area': area}

            fields=[images, aw, ah, w, h, normed_w, normed_h, area]

            with open(r'processed_vals.csv', 'a') as f:
                writer = csv.writer(f)
                writer.writerow(fields)
    except:
        print("Noted image" + images)
        continue